## How to use Pandas Groupby-Transform duet

In this notebook we will illustrate what is pandas transform functions, and how to use it.

In [1]:
import pandas as pd

### Generate dummy dataframe 

Here we are creating a dummy dataframe. We assume that a customer can have n orders, an order can have m items, and each item can be ordered x times

In [2]:
# Create a dummy dataframe
orders_df = pd.DataFrame()
orders_df['customer_id'] = [1,1,1,1,1,2,2,3,3,3,3,3]
orders_df['order_id'] = [1,1,1,2,2,3,3,4,5,6,6,6]
orders_df['item'] = ['apples', 'chocolate', 'chocolate', 'coffee', 'coffee', 'apples', 
                     'bananas', 'coffee', 'milkshake', 'chocolate', 'strawberry', 'strawberry']

____________

### Example 1 : Count the number of orders per customer

There are two main parts for the transforing a column in pandas. First, decide on the logic of the transformation. We prefer to implement this in a seperate function outside the pipe. This way we can easily develop and debug the function separately, and then apply it to the dataframe.  
. 

First, we define the **function** that will be applied per customer_id


In [3]:
count_number_of_orders = lambda x: len(x.unique())

Now, we can **tranform** each group using the logic defined above

In [4]:
orders_df['number_of_orders_per_cient'] = (               
                     orders_df                            # Take the original dataframe
                    .groupby(['customer_id'])['order_id'] # Create a group for each customer_id 
                                                          # & select the order_id
                    .transform(count_number_of_orders))   # Apply the function to each group  

Inspecting the **results** ... 

In [5]:
orders_df

,customer_id,order_id,item,number_of_orders_per_cient
0,1,1,apples,2
1,1,1,chocolate,2
2,1,1,chocolate,2
3,1,2,coffee,2
4,1,2,coffee,2
5,2,3,apples,1
6,2,3,bananas,1
7,3,4,coffee,3
8,3,5,milkshake,3
9,3,6,chocolate,3


____________

### Example 2 : Check if the items were ordered more than once in each orders

In the previous example, we had one result per client. However, functions returning different values for the group can also be applied.
We will follow the same paradigm of defining the function, then transforming the Series. 
.   

First, we define the **function** that will be applied per group

In [6]:
def multiple_items_per_order(_items):
    # Apply .duplicated, which will return True is the item occurs more than once.
    multiple_item_bool = _items.duplicated(keep=False) 
    return(multiple_item_bool)

Now, we can **tranform** each group using the logic defined above

In [7]:
orders_df['item_duplicated_per_order'] = (                    
                        orders_df                             # Take the orders dataframe
                        .groupby(['order_id'])['item']        # Create a group for each order_id 
                                                              # & select the item
                        .transform(multiple_items_per_order)) # Apply the defined function to each group

Inspecting the **results** ... 

In [8]:
orders_df

,customer_id,order_id,item,number_of_orders_per_cient,item_duplicated_per_order
0,1,1,apples,2,False
1,1,1,chocolate,2,True
2,1,1,chocolate,2,True
3,1,2,coffee,2,True
4,1,2,coffee,2,True
5,2,3,apples,1,False
6,2,3,bananas,1,False
7,3,4,coffee,3,False
8,3,5,milkshake,3,False
9,3,6,chocolate,3,False


___________________________

## How are the transform and aggregate functions related?

In the case that the transformation function is returning the same result for the group (check the first example), the `transform` function is comparable to an `aggeregate` function followed by a `merge`.

Let's try to count the number of orders per customer (first example) using `aggregate`.

. 

First, we define the **function**


In [9]:
count_number_of_orders = lambda x: len(x.unique())

Now, we can **aggregate** each group using the logic defined above.
Note that the aggregate function is returning a single result per group

In [10]:
n_orders_using_agg = (
             orders_df                             # Take the original dataframe
            .groupby(['customer_id'])['order_id']  # Create a group for each customer_id 
                                                   # & select the order_id
            .aggregate(count_number_of_orders))    # Aggregate each group using the defined fn

n_orders_using_agg

customer_id
1    2
2    1
3    3
Name: order_id, dtype: int64

Let's try to **merge** that to the orders dataframe

In [11]:
# Prepare series as a dataframe
n_orders_using_agg.name = 'n_orders_aggregate_fn'
n_orders_using_agg = n_orders_using_agg.reset_index() # Set the index as a column

In [12]:
# Merge with the original dataframe
pd.merge(orders_df, n_orders_using_agg, how = 'outer' )

,customer_id,order_id,item,number_of_orders_per_cient,item_duplicated_per_order,n_orders_aggregate_fn
0,1,1,apples,2,False,2
1,1,1,chocolate,2,True,2
2,1,1,chocolate,2,True,2
3,1,2,coffee,2,True,2
4,1,2,coffee,2,True,2
5,2,3,apples,1,False,1
6,2,3,bananas,1,False,1
7,3,4,coffee,3,False,3
8,3,5,milkshake,3,False,3
9,3,6,chocolate,3,False,3


___________________________